In [1]:
from laserembeddings import Laser

laser = Laser()

# if all sentences are in the same language:

embeddings = laser.embed_sentences(
    ['let your neural network be polyglot',
     'biarkan jaringan saraf Anda menjadi poliglot'],
    lang=['en', 'id', 'in'])  # lang is only used for tokenization

# embeddings is a N*1024 (N = number of sentences) NumPy array
print(embeddings)

[[0.00156426 0.01067378 0.00388247 ... 0.02179701 0.0062856  0.01434554]
 [0.00023719 0.00806764 0.01312093 ... 0.00290587 0.01923944 0.02042993]]


In [2]:
from scipy import spatial
dist_1 = spatial.distance.cosine(embeddings[0], embeddings[1])
dist_1

0.11816513538360596

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_colwidth', None)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
from gensim.models import Word2Vec
import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer('english')

from numpy import dot
from numpy.linalg import norm

/Users/sibish/Documents/OMSCS/CS 7643/env/laser/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package wordnet to /Users/sibish/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
train_df = pd.read_csv('train.csv')
tmp = train_df.groupby('label_group').posting_id.agg('unique').to_dict()
train_df['target'] = train_df.label_group.map(tmp)
train_df
list = train_df[train_df['posting_id'] == 'train_129225211']['target'][0]
list = ' '.join(list)
list
def target_string(list):
    return ' '.join(list)
    
train_df['target_string'] = train_df.apply(lambda x: target_string(x['target']),axis=1)
train_df.head(2)

,posting_id,image,image_phash,title,label_group,target,target_string
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,"[train_129225211, train_2278313361]",train_129225211 train_2278313361
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE",2937985045,"[train_3386243561, train_3423213080]",train_3386243561 train_3423213080


In [15]:
train_df_val = train_df.head(1000)

In [16]:
embeddings_list = []

In [17]:
for match_index, match_row in train_df.iterrows():
    #print(match_row['title']),
    embeddings = laser.embed_sentences(
     [match_row['title']],
     lang=['en', 'id', 'in']),
    #print(type(embeddings[0])),
    embeddings_list.append(embeddings[0])

In [53]:
def find_similarity(embedding1, embedding2):
    dist_1 = spatial.distance.cosine(embedding1, embedding2)
    return dist_1

In [ ]:
for match_index, match_row in train_df_val.iterrows():
    print(match_row['title'])
    match_title=match_row['title']
    match_embedding = embeddings_list[match_index]
    matched = ""
    for index, row in train_df.iterrows():
        current_title=row['title']
        current_embedding = embeddings_list[index]
        sim = find_similarity(match_embedding, current_embedding )
        if sim < 0.125:
            print('matched::',match_title, current_title, sim)
            if len(matched)< 1:
              matched = row['posting_id']
            else:
              matched = matched + " " + row['posting_id']
    print("matched ", matched)  
    train_df_val.loc[match_index,'predicted'] = matched

Paper Bag Victoria Secret
matched:: Paper Bag Victoria Secret Paper Bag Victoria Secret 0.0
matched:: Paper Bag Victoria Secret PAPER BAG VICTORIA SECRET 0.0
matched  train_129225211 train_2278313361
Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE
matched:: Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE 0.0
matched:: Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE 3M VHB Double Tape Automotive 4900 tebal 1.1 mm size 12mm x 4.5m - 1 Pcs - Merah 0.11508774757385254


/Users/sibish/Documents/OMSCS/CS 7643/env/laser/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


matched:: Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE DOUBLE TAPE BUSA 3M Pe Foam Tape 24mm x 4M ORIGINAL 0.11750096082687378
matched:: Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE Double Tape VHB 3M ORIGINAL 12mm x 4.5mm Busa Perekat 0.05982929468154907
matched:: Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE DOUBLE TAPE BUSA 3M Pe Foam Tape 24mm x 4M ORIGINAL 0.11750096082687378
matched:: Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE DOUBLE TAPE BUSA 3M Pe Foam Tape 24mm x 4M ORIGINAL 0.11750096082687378
matched:: Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE Double Tape 3M 4900 VHB 24 mm x 4.5m tebal 1.1 mm 24mm Ori Original 24 mm 1 inchi 0.11110645532608032
matched  train_3386243561 train_2120597446 train_860027362 train_3423213080 train_211200223 train_1974987067 train_1831941588
Maling TTS Canned Pork Luncheon Meat 397 gr
matched:: Maling TTS Canned Pork Luncheon Meat 397 gr Maling TTS Canned Pork Lunche

matched  train_999391364 train_4285884352
Kangaroo Teflon / Allu Fry Pan 18 cm - KG652
matched:: Kangaroo Teflon / Allu Fry Pan 18 cm - KG652 Kangaroo Teflon / Allu Fry Pan 18 cm - KG652 0.0
matched:: Kangaroo Teflon / Allu Fry Pan 18 cm - KG652 Maxim Teflon / Fry Pan Halania 18 cm - HAL18FP 0.09779036045074463
matched:: Kangaroo Teflon / Allu Fry Pan 18 cm - KG652 Maxim Teflon / Fry Pan Halania 20 cm - HAL20FP 0.1029401421546936
matched:: Kangaroo Teflon / Allu Fry Pan 18 cm - KG652 Maxim Teflon Valentino / Fry Pan 18 cm 0.12231689691543579
matched  train_2238403912 train_2591650844 train_2709375315 train_2622833587
CHOCO BALL LAGIE COKLAT Lagie Grosir Cokelat Lagie Murah Chocoball Kiloan Chocobal Coklat Kiloan 1Kg
matched:: CHOCO BALL LAGIE COKLAT Lagie Grosir Cokelat Lagie Murah Chocoball Kiloan Chocobal Coklat Kiloan 1Kg CHOCO BALL LAGIE COKLAT Lagie Grosir Cokelat Lagie Murah Chocoball Kiloan Chocobal Coklat Kiloan 1Kg 0.0
matched  train_1396146683
[INCLUDE PAJAK] NCT DREAM RELOAD

matched:: Pelindung Lutut Bayi | Kaos Kaki Baby Knee Pad Protector [SEPASANG] KAOS KAKI PELINDUNG LUTUT BAYI / KNEE PROTECTOR BAYI 0.11422330141067505
matched:: Pelindung Lutut Bayi | Kaos Kaki Baby Knee Pad Protector Pelindung Lutut Bayi Knee Protector 0.07053077220916748
matched:: Pelindung Lutut Bayi | Kaos Kaki Baby Knee Pad Protector Pelindung lutut bayi baby kned protector 0.11854928731918335
matched  train_4193869534 train_2796119577 train_1109958472 train_3860005169
SOMEBYMI AHA-BHA-PHA 30 DAYS MIRACLE ACNE CLEAR BODY CLEANSER 400ML
matched:: SOMEBYMI AHA-BHA-PHA 30 DAYS MIRACLE ACNE CLEAR BODY CLEANSER 400ML SOMEBYMI AHA-BHA-PHA 30 DAYS MIRACLE ACNE CLEAR BODY CLEANSER 400ML 0.0
matched:: SOMEBYMI AHA-BHA-PHA 30 DAYS MIRACLE ACNE CLEAR BODY CLEANSER 400ML SOMEBYMI Aha Bha Pha 30 Days Miracle Toner 150ml 0.12480425834655762
matched:: SOMEBYMI AHA-BHA-PHA 30 DAYS MIRACLE ACNE CLEAR BODY CLEANSER 400ML Somebymi AHA-BHA-PHA 30 Days Miracle Serum 50ml 0.11204183101654053
matched  t

...

matched:: missha airy fit sheet mask [BPOM] MISSHA Airy Fit Sheet Mask | Original 0.10125523805618286
matched:: missha airy fit sheet mask [BPOM] Missha Airy Fit Sheet Mask 19g 0.12156718969345093
matched  train_496484454 train_505220615 train_3855605518 train_2529762995 train_2388251788 train_1665508579 train_1689298260 train_3572766330
SEPATU RUNNING ADIDAS NEO THICK LINE ORIGINAL BOX
matched:: SEPATU RUNNING ADIDAS NEO THICK LINE ORIGINAL BOX SEPATU RUNNING ADIDAS NEO THICK LINE ORIGINAL BOX 0.0
matched  train_2415058924
Sasa LaRasa Bumbu Lumur Ayam Spesial isi 10 @26gr (RENCENG)
matched:: Sasa LaRasa Bumbu Lumur Ayam Spesial isi 10 @26gr (RENCENG) Sasa LaRasa Bumbu Lumur Ayam Spesial isi 10 @26gr (RENCENG) 0.0
matched  train_685198319
TERLARIS Tuffah Nail Henna Kutek kuku 10ml sah sholat agen Resmi
matched:: TERLARIS Tuffah Nail Henna Kutek kuku 10ml sah sholat agen Resmi TERLARIS Tuffah Nail Henna Kutek kuku 10ml sah sholat agen Resmi 0.0
matched  train_2969481517
Bergo anak 0-3 t

matched:: CARVIL Sandal Pria Sponge Jiran-M (BROWN, GREY, BLACK) Carvil Sandal Sponge Pria Jiran-M Brown 0.10296130180358887
matched  train_57281679 train_4214189853
LC Adaptor memori Memory / adapter micro sd
matched:: LC Adaptor memori Memory / adapter micro sd LC Adaptor memori Memory / adapter micro sd 0.0
matched:: LC Adaptor memori Memory / adapter micro sd Micro SD Adapter / Adaptor / Adaptor MicroSd / Micro SD / MMC / Kartu Memori 0.10646289587020874
matched:: LC Adaptor memori Memory / adapter micro sd ADAPTOR MEMORY CARD RUMAH MMC ADAPTOR MICRO SD ADAPTER SD CARD 0.11301213502883911
matched  train_3041799771 train_3849802991 train_1241055511
EAGLE\nEucalyptus\nDisinfectant Spray Disinfektan Spray
matched:: EAGLE\nEucalyptus\nDisinfectant Spray Disinfektan Spray EAGLE\nEucalyptus\nDisinfectant Spray Disinfektan Spray 0.0
matched  train_78020174
MITO ELECTRIC OVEN FANTASY MO-888
matched:: MITO ELECTRIC OVEN FANTASY MO-888 MITO ELECTRIC OVEN FANTASY MO-888 0.0
matched:: MITO ELE

matched:: \xe2\x9c\xbf GOBABYSHOP \xe2\x9c\xbf PROMO CETAPHIL!! Cetaphil Gentle Skin Cleanser 500ml CETAPHIL GENTLE SKIN CLEANSER 500 ML 0.12427020072937012
matched:: \xe2\x9c\xbf GOBABYSHOP \xe2\x9c\xbf PROMO CETAPHIL!! Cetaphil Gentle Skin Cleanser 500ml Cetaphil Gentle Skin Cleanser 500 ml 0.12427020072937012
matched:: \xe2\x9c\xbf GOBABYSHOP \xe2\x9c\xbf PROMO CETAPHIL!! Cetaphil Gentle Skin Cleanser 500ml Cetaphil Gentle Skin Cleanser 500ml 0.11087667942047119
matched:: \xe2\x9c\xbf GOBABYSHOP \xe2\x9c\xbf PROMO CETAPHIL!! Cetaphil Gentle Skin Cleanser 500ml [PROMO] Cetaphil Gentle Skin Cleanser 500 ML 0.12443387508392334
matched  train_1480884871 train_4062110244 train_77384994 train_3064066145 train_567083000
Wardah Refill Instaperfect MATTE FIT Powder Foundation 14. CREME 13 g
matched:: Wardah Refill Instaperfect MATTE FIT Powder Foundation 14. CREME 13 g Wardah Refill Instaperfect Matte Fit Powder Foundation (100% Original) 0.11942851543426514
matched:: Wardah Refill Instaperf

matched  train_3761611877
For Samsung Galaxy A51 A71 M30s A70 A50 A50S A30S A30 A20 A40 A10 Case Back Cover Soft Silicone Case
matched:: For Samsung Galaxy A51 A71 M30s A70 A50 A50S A30S A30 A20 A40 A10 Case Back Cover Soft Silicone Case Samsung Galaxy A10 A20 A30 A40 A50 A70 M10 M20 M30 Silicone Soft Matte Phone Case Casing Back Cover 0.07903456687927246
matched:: For Samsung Galaxy A51 A71 M30s A70 A50 A50S A30S A30 A20 A40 A10 Case Back Cover Soft Silicone Case For Samsung Galaxy A51 A71 M30s A70 A50 A50S A30S A30 A20 A40 A10 Case Back Cover Soft Silicone Case 0.0
matched:: For Samsung Galaxy A51 A71 M30s A70 A50 A50S A30S A30 A20 A40 A10 Case Back Cover Soft Silicone Case Soft Case Simple Warna Polos untuk Samsung A51 a71 a10s a20s a50s A10 M10 A70 a50s A10 M10 J2 J7 0.11727863550186157
matched:: For Samsung Galaxy A51 A71 M30s A70 A50 A50S A30S A30 A20 A40 A10 Case Back Cover Soft Silicone Case SAMSUNG M10 M20 A10 A20 A30 A50 A70 A30S A50S / ANTI CRACK FUZE BENING SAMSUNG / AKRILI

matched  train_1197007619
Tombow Dual Brush Pen Light Ochre
matched:: Tombow Dual Brush Pen Light Ochre Tombow Dual Brush Pen Light Ochre 0.0
matched:: Tombow Dual Brush Pen Light Ochre NEW ! Tombow Dual Brush Pen Bright Pallete Set 10 CLR 0.11524403095245361
matched:: Tombow Dual Brush Pen Light Ochre Tombow Dual Brush Pen Set 10 Colors Bright Palette Murah Hemat 0.11113810539245605
matched:: Tombow Dual Brush Pen Light Ochre Tombow Dual Brush Pen Avocado 0.10365033149719238
matched:: Tombow Dual Brush Pen Light Ochre Tombow Dual Brush Pen Yellow Gold 0.07901167869567871
matched:: Tombow Dual Brush Pen Light Ochre Tombow Dual Brush Pen Set 10 - Bright Palette 0.11150979995727539
matched:: Tombow Dual Brush Pen Light Ochre Tombow Dual Brush Pen Set 10 - Bright Palette 0.11150979995727539
matched  train_2761157068 train_319000217 train_3506259608 train_701905731 train_1927506862 train_3068962481 train_4029724154
Kumpulan Doa Mustajab Dan Dzikir Pilihan [ Darul Haq - al Qahtani ] Riniaga

In [ ]:
train_df_val

In [ ]:
def match(actual, predicted):
    if actual == predicted:
        return True
    else:
        return False
    
train_df_val['prediction_accuracy'] = train_df_val.apply(lambda x: match(x['target_string'], x['predicted']),axis=1)

In [ ]:
prediction_accuracy_v1 = train_df_val.groupby(['prediction_accuracy'])['posting_id'].count()

In [ ]:
prediction_accuracy_v1

In [ ]:
#Accuracy
print("Accuracy of V5" , (prediction_accuracy_v1[True]/(prediction_accuracy_v1[True] + prediction_accuracy_v1[False]))*100)

